In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE51495"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE51495"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE51495.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE51495.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE51495.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral blood mononuclear cell- and cortical bone-derived transcriptional profiles"
!Series_summary	"Large-scale transcriptional profiling has enormous potential for discovery of osteoporosis susceptibility genes and for identification of the molecular mechanisms by which these genes and associated pathways regulate bone maintenance and turnover. A potential challenge in the use of this method for the discovery of osteoporosis genes is the difficulty of obtaining bone tissue samples from large numbers of individuals. In this study, we tested the applicability of using peripheral blood mononuclear cell (PBMC)-derived transcriptional profiles as a surrogate to cortical bone transcriptional profiles to address questions of skeletal genetics. We used a well-established and genetically well-characterized nonhuman primate model for human bone maintenance and turnover. We determined that a high degree of overlap exists in gene expression of cortical b

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from PBMCs and cortical bone
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Osteoporosis): No direct trait information in the sample characteristics, but we can infer from tissue type
# Row 2 contains tissue information, which can be used to distinguish bone samples from blood samples
trait_row = 2

# For age: Row 1 contains age information
age_row = 1

# For gender: Row 0 contains gender information
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert tissue type to binary trait value (bone sample = 1, blood sample = 0)"""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # In this dataset, we're interested in bone samples vs. blood samples
    # Here we assign 1 to bone samples since they're directly related to osteoporosis
    if "cortical bone" in value.lower():
        return 1
    elif "blood" in value.lower() or "pbmc" in value.lower():
        return 0
    return None

def convert_age(value):
    """Convert age string to floating point number"""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        # Extract digits from the age string
        age_value = float(re.search(r'\d+\.\d+|\d+', value).group())
        return age_value
    except (ValueError, AttributeError):
        return None

def convert_gender(value):
    """Convert gender string to binary (female = 0, male = 1)"""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save Metadata - Initial Filtering
# Trait data is available based on our inference from tissue type
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the sample characteristics data from the parent directory
    # We need to access the clinical data that was extracted in a previous step
    try:
        # Assuming clinical_data is already in memory from a previous step
        print("Available variables for this step:", locals().keys())
        
        # Since clinical_data might not be pre-loaded, let's try to import it
        # or construct it from available information
        import os
        import pandas as pd
        import re
        
        # Check if the variable 'clinical_data' is defined in the previous step
        if 'clinical_data' not in locals():
            # We need to construct clinical_data based on the sample characteristics
            # Using the information provided in the previous step output
            sample_chars = {
                0: ['gender: female', 'gender: male'],
                1: ['age (yrs): 11.24', 'age (yrs): 14.15', 'age (yrs): 14.03', 'age (yrs): 19.16', 
                    'age (yrs): 16.66', 'age (yrs): 18.26', 'age (yrs): 17.59', 'age (yrs): 12.51', 
                    'age (yrs): 13.53', 'age (yrs): 12.06', 'age (yrs): 15.08', 'age (yrs): 14.46', 
                    'age (yrs): 20.18', 'age (yrs): 21.95', 'age (yrs): 27.34'],
                2: ['tissue: Baboon cortical bone', 'tissue: Baboon Peripheral blood mononuclear cells']
            }
            
            # Create a DataFrame from the sample characteristics
            clinical_data = pd.DataFrame(sample_chars)
            
        # Extract clinical features
        selected_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(selected_features)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the extracted clinical features
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        print("Skipping clinical feature extraction due to data access issues.")


Available variables for this step: dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'sys', 'os', 'gzip', 'io', 'json', 're', 'Callable', 'Optional', 'List', 'Tuple', 'Dict', 'Union', 'Any', 'pd', 'geo_get_relevant_filepaths', 'tcga_get_relevant_filepaths', 'line_generator', 'filter_content_by_prefix', 'get_background_and_clinical_data', 'get_gene_annotation', 'get_gene_mapping', 'get_genetic_data', 'extract_human_gene_symbols', 'apply_gene_mapping', 'normalize_gene_symbols_in_index', 'get_feature_data', 'judge_binary_variable_biased', 'judge_continuous_variable_biased', 'tcga_convert_trait', 'tcga_convert_gender', 'tcga_convert_age', 'get_unique_values_by_row', 'tcga_select_clinical_features', 'geo_select_clinical_features', 'geo_link_clinical_genetic_data', 'handle_missing_values', 'judge_and_remove_biased_featu

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343292', 'ILMN_1343293', 'ILMN_1343294',
       'ILMN_1651209', 'ILMN_1651217', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651234', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237',
       'ILMN_1651238', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260',
       'ILMN_1651261', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1698220', 'ILMN_1810835', 'ILMN_1782944', 'ILMN_1692858', 'ILMN_1668162'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_13666', 'ILMN_10478', 'ILMN_27850', 'ILMN_10309', 'ILMN_7652'], 'Transcript': ['ILMN_13666', 'ILMN_175835', 'ILMN_27850', 'ILMN_10309', 'ILMN_7652'], 'ILMN_Gene': ['PHTF2', 'SPRR3', 'GPR37L1', 'FBXO25', 'DGAT2L3'], 'Source_Reference_ID': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'RefSeq_ID': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'Entrez_Gene_ID': [57157.0, 6707.0, 9283.0, 26260.0, 158833.0], 'GI': [40254932.0, 4885606.0, 31377792.0, 34878756.0, 61888901.0], 'Accession': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'Symbol': ['PHTF2', 'SPRR3', 'GPR37L1', 'FBXO25', 'DGAT2L3'], 'Protein_Prod

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in gene annotation for mapping
# Looking at the previews from previous steps:
# - Gene expression data uses 'ILMN_*' identifiers (Illumina probe IDs)
# - Gene annotation dataframe has 'ID' column with same format identifiers
# - 'Symbol' column contains the gene symbols we need

# 2. Get the gene mapping dataframe
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'Symbol'  # Column containing gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# The apply_gene_mapping function handles the many-to-many relationships correctly
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few rows of the mapped gene expression data
print("Gene expression data after mapping to gene symbols:")
print(gene_data.shape)
print(gene_data.index[:20])


Gene expression data after mapping to gene symbols:
(17609, 30)
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS',
       'AACS', 'AADAC', 'AADACL1', 'AADACL2', 'AADACL4', 'AADAT', 'AAK1',
       'AAMP', 'AANAT', 'AARS', 'AARS2', 'AARSD1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="This is an HPV-transformed keratinocyte cell line study focusing on transformation stages: 1 for anchorage independent (more advanced cancer stage), 0 for earlier stages."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE51495.csv
Clinical data saved to ../../output/preprocess/Osteoporosis/clinical_data/GSE51495.csv
Linked data shape: (30, 16994)


For the feature 'Osteoporosis', the least common label is '1.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Osteoporosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 13.655
  50% (Median): 15.08
  75%: 18.935000000000002
Min: 11.24
Max: 27.34
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 2 occurrences. This represents 6.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Osteoporosis/GSE51495.csv
